# 🗺️ SAT POI Viewer with WD + OSM
This notebook loads POIs (e.g. toilets, water points) from Wikidata with optional OSM matches and renders them on a Folium map with filtering options.

In [1]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    

Start: 2025-07-24 19:21:49


In [5]:
    qid = res["item"]["value"].split('/')[-1]
    wd_url = f"https://www.wikidata.org/wiki/{qid}"
    data.append({"qid": qid, "label": label, "lat": lat, "lon": lon, "source": source, "osm_url": osm_url, "wd_url": wd_url})
    # Run a live SPARQL query to get SAT POIs with optional OSM links
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
query = """
SELECT ?item ?itemLabel ?coord ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P6104 wd:Q134294510; wdt:P625 ?coord.
  OPTIONAL { ?item wdt:P10689 ?OSMway. }
  OPTIONAL { ?item wdt:P402 ?OSMrel. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'sv,en'. }
}
"""

sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

import re

data = []
for res in results["results"]["bindings"]:    
     label = res["itemLabel"]["value"]
     coord = res["coord"]["value"]  # e.g. Point(18.858 59.756)
     lon, lat = map(float, re.findall(r"[-+]?\d+\.\d+", coord))
     node = res.get("OSMnode", {}).get("value")
     way = res.get("OSMway", {}).get("value")
     rel = res.get("OSMrel", {}).get("value")

     if node:
         source = "OSM"
         osm_url = f"https://www.openstreetmap.org/node/{node}"
     elif way:
         source = "OSM"
         osm_url = f"https://www.openstreetmap.org/way/{way}"
     elif rel:
         source = "OSM"
         osm_url = f"https://www.openstreetmap.org/relation/{rel}"
     else:
         source = "WD"
         osm_url = None

     if node or way or rel:
         source = "both"

     data.append({"label": label, "lat": lat, "lon": lon, "source": source, "osm_url": osm_url})

df = pd.DataFrame(data)
df.head()

NameError: name 'res' is not defined

In [ ]:
import folium
import pandas as pd
from folium import FeatureGroup
from shapely.geometry import Point
from IPython.display import display

# Sample POIs (normally loaded via SPARQL)
data = [
    {"label": "Toilet WD", "lat": 59.75, "lon": 18.85, "source": "WD", "osm_url": None},
    {"label": "Toilet OSM", "lat": 59.752, "lon": 18.855, "source": "OSM", "osm_url": "https://www.openstreetmap.org/node/123"},
    {"label": "Water both", "lat": 59.76, "lon": 18.82, "source": "both", "osm_url": "https://www.openstreetmap.org/node/456"},
]
df = pd.DataFrame(data)

In [ ]:
    popup_html = f"""
        <b>{row['label']}</b><br>
        <a href='{row['wd_url']}' target='_blank'>Wikidata: {row['qid']}</a><br>
        {'<a href="' + row['osm_url'] + '" target="_blank">OSM link</a>' if row['osm_url'] else ''}
    """
    marker = folium.Marker(
        [row["lat"], row["lon"]],
        popup=popup_html,
        icon=folium.Icon(color="blue" if row["source"] == "WD" else "green" if row["source"] == "OSM" else "purple")
    )

In [ ]:
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))